In [8]:
import gdal
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import torch
from glob import glob
import shapely
import random
from tqdm import tqdm
from collections import Counter

# 转换图像为Patch

In [4]:
def read_img(path):
    ds=gdal.Open(path)
    trf=ds.GetGeoTransform()
    img=ds.ReadAsArray()
    return {'trf':trf,'img':img}
def get_patch(data,size):
    img=data['img']
    trf=data['trf']
    shape=img.shape
    img=np.pad(img,((0,0),(0,size-shape[1]%size),(0,size-shape[2]%size)),'constant',constant_values=np.nan)
    shape=img.shape
    img=img.reshape([shape[0],int(shape[1]/size),-1,shape[2]])
    shape=img.shape
    img=img.reshape([shape[0],shape[1],shape[2],int(shape[3]/size),-1])
    return {'img':img.transpose((0,1,3,2,4)),'trf':(trf[0]-trf[1]/2,trf[1]*size,trf[2],trf[3]-trf[5]/2,trf[4],trf[5]*size)}
size=6
keys=['A','B','C','D']
dirs=['Livoberezhyny_band2348_v2','Livoberezhyny_bandother_v2']
root='/home/hk/python_script/Ukraine/data/img/Livoberezhyny/'
for dir in dirs:
    for key in keys:
        if dir.split('_')[1]=='bandother':
            size_=int(size/2)
        else:
            size_=int(size)
        path=glob(os.path.join(root+dir,key+'*'))[0]
        data=read_img(path)
        print(size_)
        print(data['img'].shape)
        data=get_patch(data,size_)
        print(data['img'].shape)
        print(data['trf'])
        data['readme']='trf分别为patch左上角坐标（不是左上角像元的中心坐标）和patch的长度'
        torch.save(data,path.replace('.tif',f'_size_{size_}.pth.tar'))


6
(4, 698, 2254)
(4, 117, 376, 6, 6)
(37.493209592117495, 0.0005389891704717128, 0.0, 47.146325972209056, 0.0, -0.0005389891704717128)
6
(4, 698, 2254)
(4, 117, 376, 6, 6)
(37.493209592117495, 0.0005389891704717128, 0.0, 47.146325972209056, 0.0, -0.0005389891704717128)
6
(4, 698, 2254)
(4, 117, 376, 6, 6)
(37.493209592117495, 0.0005389891704717128, 0.0, 47.146325972209056, 0.0, -0.0005389891704717128)
6
(4, 698, 2254)
(4, 117, 376, 6, 6)
(37.493209592117495, 0.0005389891704717128, 0.0, 47.146325972209056, 0.0, -0.0005389891704717128)
3
(6, 349, 1127)
(6, 117, 376, 3, 3)
(37.49325450788171, 0.0005389891704717128, 0.0, 47.14637088797326, 0.0, -0.0005389891704717128)
3
(6, 349, 1127)
(6, 117, 376, 3, 3)
(37.49325450788171, 0.0005389891704717128, 0.0, 47.14637088797326, 0.0, -0.0005389891704717128)
3
(6, 349, 1127)
(6, 117, 376, 3, 3)
(37.49325450788171, 0.0005389891704717128, 0.0, 47.14637088797326, 0.0, -0.0005389891704717128)
3
(6, 349, 1127)
(6, 117, 376, 3, 3)
(37.49325450788171, 0.00

# 制作patch的label

In [12]:
def get_dam_mask(point,patch,weight):
    trf=patch['trf']
    p_coor=[shapely.geometry.mapping(list(point)[i])['coordinates'] for i in range(len(point))]
    mask=np.zeros((patch['img'].shape[1],patch['img'].shape[2]))
    p_x_y=[[int((i[0]-trf[0])/trf[1]),int((i[1]-trf[3])/trf[5])] for i in p_coor]
    for i in p_x_y:
        mask[i[1],i[0]]+=weight
    return mask
def get_masks(point_file=glob('../data/UNOSAT/LivoberezhnyiandZhovtnevyi_0507-08-12/*DA*.shp')[0],
            label_code='B-C',
            patch=torch.load('/home/hk/python_script/Ukraine/data/img/Livoberezhyny/Livoberezhyny_band2348_v2/A-2021-6-1-2021-10-1_size_6.pth.tar')):
    damage_L_gpd=gpd.read_file(point_file)
    print(len(damage_L_gpd))
    if label_code=='B-C':
        key='d_Main_Dam'
        date='三月'
    elif label_code=='C-D':
        key='d_Main_D_1'
        date='五月'
    point_num={}
    point=damage_L_gpd[damage_L_gpd[key]=='Destroyed']['geometry']
    point_num['destroyed']=len(point)
    print('destroyed:',len(point))
    mask_destroyed=get_dam_mask(point,patch,1)
    point=damage_L_gpd[damage_L_gpd[key]=='Severe Damage']['geometry']
    point_num['severe']=len(point)
    print('Severe Damage:',len(point))
    mask_severe=get_dam_mask(point,patch,1)
    point=damage_L_gpd[damage_L_gpd[key]=='Moderate Damage']['geometry']
    point_num['moderate']=len(point)
    print('Moderate Damage:',len(point))
    mask_moderate=get_dam_mask(point,patch,1)
    point=damage_L_gpd[damage_L_gpd[key]=='Possible Damage']['geometry']
    point_num['possible']=len(point)
    print('May Damage:',len(point))
    mask_may=get_dam_mask(point,patch,1)
    mask=mask_destroyed*6+mask_severe*3+mask_moderate
    print(f'{date}得分',dict(Counter(list(mask.flatten()))))
    mask_type=np.zeros(mask.shape)
    mask_type[mask>0]=1
    mask_type[mask>=6]=2
    mask_type[mask_may==1]=-1
    print(f'{date}损毁类型',Counter(list(mask_type.flatten())))
    all_mask=[mask_destroyed,mask_severe,mask_moderate,mask_may,mask,mask_type]
    info='6层分别为完全 严重 中等 可能损毁 以及损毁得分和类型\n得分:完全算6分 严重算3分 中等算1分 其他0分\n类型0:得分0 类型1:得分1-5 类型2:得分大于6 类型-1:可能损毁但视觉不明显'
    result={'info':info,'point_num':point_num,'mask':all_mask}
    return result
all_mask_BC=get_masks()
all_mask_CD=get_masks(label_code='C-D')
torch.save(all_mask_BC,os.path.join('/home/hk/python_script/Ukraine/data/sample/sample_sentinel2/','mask_BC.pth.tar'))
torch.save(all_mask_CD,os.path.join('/home/hk/python_script/Ukraine/data/sample/sample_sentinel2/','mask_CD.pth.tar'))

5660
destroyed: 61
Severe Damage: 300
Moderate Damage: 311
May Damage: 68
三月得分 {0.0: 43442, 3.0: 203, 6.0: 50, 1.0: 229, 4.0: 14, 9.0: 12, 5.0: 2, 2.0: 15, 7.0: 11, 8.0: 3, 13.0: 1, 10.0: 2, 16.0: 1, 14.0: 1, 12.0: 5, 11.0: 1}
三月损毁类型 Counter({0.0: 43382, 1.0: 459, 2.0: 87, -1.0: 64})
5660
destroyed: 315
Severe Damage: 2132
Moderate Damage: 3002
May Damage: 194
五月得分 {0.0: 40560, 3.0: 768, 6.0: 222, 1.0: 1501, 2.0: 223, 12.0: 57, 19.0: 5, 13.0: 18, 18.0: 11, 7.0: 102, 4.0: 223, 15.0: 20, 9.0: 78, 5.0: 79, 28.0: 2, 14.0: 8, 11.0: 10, 10.0: 44, 8.0: 35, 16.0: 13, 27.0: 2, 21.0: 1, 24.0: 2, 17.0: 5, 33.0: 1, 22.0: 2}
五月损毁类型 Counter({0.0: 40429, 1.0: 2755, 2.0: 628, -1.0: 180})


# 制作样本

In [99]:
label_code=['BC','CD']
pre_code=['A']
post_code=['B','C','D']
img_dir=['/home/hk/python_script/Ukraine/data/img/Livoberezhyny/Livoberezhyny_band2348_v2/',
        '/home/hk/python_script/Ukraine/data/img/Livoberezhyny/Livoberezhyny_bandother_v2/']
label_dir='/home/hk/python_script/Ukraine/data/sample/sample_sentinel2/'
label={}
for i in label_code:
        label[i]=torch.load(os.path.join(label_dir,f'mask_{i}.pth.tar'))
pre_path={}
for i in pre_code:
        pre_path[i]=[glob(os.path.join(j,i+'*.pth.tar'))[0] for j in img_dir]
post_path={}
for i in post_code:
        post_path[i]=[glob(os.path.join(j,i+'*.pth.tar'))[0] for j in img_dir]
def mask_samples(pre_path,post_path,label):
        sample_p1=[]
        sample_p2=[]
        sample_n=[]
        for post_img_code in post_path.keys():
                post_img10=torch.load(post_path[post_img_code][0])['img']
                post_img20=torch.load(post_path[post_img_code][1])['img']
                pre_img10=torch.load(pre_path['A'][0])['img']
                pre_img20=torch.load(pre_path['A'][1])['img']
                before_label_code=None
                after_label_code=None
                for i in label.keys():
                        if i.find(post_img_code)==1:
                                before_label_code=i
                        if i.find(post_img_code)==0:
                                after_label_code=i

                
                #前后都有label
                if before_label_code is not None and after_label_code is not None:
                        print(before_label_code,post_img_code,after_label_code) 

                        before_label=label[before_label_code]['mask'][-1].copy()
                        after_label=label[after_label_code]['mask'][-1].copy()
                        mask=np.zeros((after_label.shape[0],after_label.shape[1]))
                        mask[before_label==after_label]=1
                        mask[before_label==-1]=0
                        mask[after_label==-1]=0
                        after_label[mask==0]=np.nan
                        sample_label=after_label
                #前有label 后没有
                elif before_label_code is not None and after_label_code is None:
                        print(before_label_code,post_img_code) 

                        before_label=label[before_label_code]['mask'][-1].copy()
                        # print(Counter(list(before_label.flatten())))
                        before_label[before_label<1.5]=np.nan
                        sample_label=before_label
                #后有label 前没有
                elif after_label_code is not None and before_label_code is None:
                        print(post_img_code,after_label_code) 

                        after_label=label[after_label_code]['mask'][-1].copy()
                        after_label[after_label>0]=-1
                        after_label[after_label<0]=np.nan
                        sample_label=after_label
                else:
                        raise()
                sample_label[np.isnan(post_img10[0,:,:,0,0])]=np.nan
                sample_label[np.isnan(post_img20[0,:,:,0,0])]=np.nan
                sample_label[np.isnan(sample_label)]=100
                sample_label.astype(np.int)
                # print(Counter(list(sample_label.flatten())))

                all_patch=[pre_img10,pre_img20,post_img10,post_img20]
                shape=post_img10.shape

                for x in range(shape[1]):
                        for y in range(shape[2]):

                                if np.isnan(all_patch[0][:,x,y]).any() and np.isnan(all_patch[1][:,x,y]).any() and np.isnan(all_patch[2][:,x,y]).any() and np.isnan(all_patch[3][:,x,y]).any():
                                        continue
                                else:
                                        if sample_label[x,y]==1:

                                                sample_p1.append([[x,y],
                                                                np.concatenate([all_patch[0][:,x,y],all_patch[2][:,x,y]],axis=0),
                                                                np.concatenate([all_patch[1][:,x,y],all_patch[3][:,x,y]],axis=0)])
                                        elif sample_label[x,y]==2:

                                                sample_p2.append([[x,y],
                                                                np.concatenate([all_patch[0][:,x,y],all_patch[2][:,x,y]],axis=0),
                                                                np.concatenate([all_patch[1][:,x,y],all_patch[3][:,x,y]],axis=0)])
                                        elif sample_label[x,y]==0:

                                                sample_n.append([[x,y],
                                                                np.concatenate([all_patch[0][:,x,y],all_patch[2][:,x,y]],axis=0),
                                                                np.concatenate([all_patch[1][:,x,y],all_patch[3][:,x,y]],axis=0)])
                
        return [sample_n,sample_p1,sample_p2]

In [108]:
sample=mask_samples(pre_path,post_path,label)
print(len(sample[0]))
print(len(sample[1]))
print(len(sample[2]))
torch.save({'info':'分别为无损毁,评分1-5损毁,评分>6损毁','sample':sample},f'../data/sample/sample_sentinel2/Livoberezhyny_class_sample_size6_Destroyed.pth.tar')

B BC
BC C CD
CD D
26210
413
701


* mask=0未破坏
* mask<6中度破坏
* mask>=6严重破坏

# 分割train和val集

In [115]:
size=6
perc=100
dir=f'../data/sample/sample_sentinel2/Livoberezhyny_size{size}_Destroyed_class'

sample_n,sample_p1,sample_p2=torch.load(f'../data/sample/sample_sentinel2/Livoberezhyny_class_sample_size{size}_Destroyed.pth.tar')['sample']

print(len(sample_n),len(sample_p1),len(sample_p2))
if not os.path.exists(dir+'/train'):
    os.makedirs(dir+'/train')
if not os.path.exists(dir+'/val'):
    os.makedirs(dir+'/val')
p1_index=list(range(len(sample_p1)))
p2_index=list(range(len(sample_p2)))
n_index=list(range(len(sample_n)))
random.shuffle(p1_index)
random.shuffle(p2_index)
random.shuffle(n_index)
p1_n=int(len(sample_p1)*2/3)
p2_n=int(len(sample_p2)*2/3)
n_n=int(len(sample_n)*2/3)
for i in tqdm(range(len(sample_p1))):
    if i<p1_n:
        torch.save(sample_p1[p1_index[i]],os.path.join(dir+'/train','P1_'+str(p1_index[i])+'.pth.tar'))
    else:
        torch.save(sample_p1[p1_index[i]],os.path.join(dir+'/val','P1_'+str(p1_index[i])+'.pth.tar'))
for i in tqdm(range(len(sample_p2))):
    if i<p2_n:
        torch.save(sample_p2[p2_index[i]],os.path.join(dir+'/train','P2_'+str(p2_index[i])+'.pth.tar'))
    else:
        torch.save(sample_p2[p2_index[i]],os.path.join(dir+'/val','P2_'+str(p2_index[i])+'.pth.tar'))
for i in tqdm(range(len(sample_n))):
    if i<n_n:
        if random.choice(range(1, 101))<=perc:
            torch.save(sample_n[n_index[i]],os.path.join(dir+'/train','N_'+str(n_index[i])+'.pth.tar'))
    else:
        if random.choice(range(1, 101))<=perc:
            torch.save(sample_n[n_index[i]],os.path.join(dir+'/val','N_'+str(n_index[i])+'.pth.tar'))

100%|██████████| 26210/26210 [00:05<00:00, 4991.25it/s]


In [116]:
len(glob('../data/sample/sample_sentinel2/Livoberezhyny_size6_Destroyed_class/train/N*'))

17473

# 删除有nan的数据

In [159]:
nan_path=[]
for i in tqdm(glob('../data/sample/sample_sentinel2/Livoberezhyny_size6_Destroyed_class/*/*')):
    data=torch.load(i)
    if np.isnan(data[1]).any() or np.isnan(data[2]).any():
        nan_path.append(i)
print(len(nan_path))

100%|██████████| 27324/27324 [00:03<00:00, 7240.28it/s]

230
